# Notebook to make a map of the currents resulting from ROMS

- Initial use is for the presentation for the ministry on April 6.
- I will use GLORYS data as it has the highest initial resolution



In [5]:
import xarray as xr
import numpy as np

In [4]:
data_type = 'GLORYS'  # 'ORA', 'GLORYS', 'SODA', 'ECCO'
run = 'old_run/complete_run'

path = f'/projects/0/einf2878/ROMS/data/{data_type}/roms_output/{run}'
data = xr.open_mfdataset(f'{path}/NorthSea4_av*.nc')

In [8]:
data.Huon

<xarray.DataArray 'Huon' (ocean_time: 2040, s_rho: 30, eta_u: 110, xi_u: 121)>
dask.array<open_dataset-b8632caffa1daff44751d6ee5f70d0cdHuon, shape=(2040, 30, 110, 121), dtype=float32, chunksize=(2040, 30, 110, 121), chunktype=numpy.ndarray>
Coordinates:
  * s_rho       (s_rho) float64 -0.9833 -0.95 -0.9167 ... -0.05 -0.01667
    lon_u       (eta_u, xi_u) float64 dask.array<chunksize=(110, 121), meta=np.ndarray>
    lat_u       (eta_u, xi_u) float64 dask.array<chunksize=(110, 121), meta=np.ndarray>
  * ocean_time  (ocean_time) datetime64[ns] 1993-01-17T12:00:00 ... 2020-12-1...
Dimensions without coordinates: eta_u, xi_u
Attributes:
    standard_name:  ocean_volume_x_transport
    long_name:      u-volume flux
    units:          meter3 second-1
    time:           ocean_time
    cell_methods:   ocean_time: mean
    grid:           grid
    location:       edge1
    field:          volume x-transport

### New idea, just use input data?

In [3]:
data = xr.open_dataset('~/NorthSea4/NorthSea4_init_GLORYS_19930116_to_20201216.nc')

In [12]:
data

<xarray.Dataset>
Dimensions:     (eta_rho: 110, xi_rho: 122, eta_u: 110, xi_u: 121, eta_v: 109, xi_v: 122, eta_psi: 109, xi_psi: 121, s_rho: 30, s_w: 31, ocean_time: 1)
Coordinates:
  * s_rho       (s_rho) float64 -0.9833 -0.95 -0.9167 ... -0.05 -0.01667
  * s_w         (s_w) float64 -1.0 -0.9667 -0.9333 ... -0.06667 -0.03333 0.0
  * ocean_time  (ocean_time) datetime64[ns] 1993-01-15
Dimensions without coordinates: eta_rho, xi_rho, eta_u, xi_u, eta_v, xi_v, eta_psi, xi_psi
Data variables: (12/24)
    lon_rho     (eta_rho, xi_rho) float64 -19.88 -19.63 -19.39 ... 9.631 9.877
    lat_rho     (eta_rho, xi_rho) float64 35.95 35.95 35.95 ... 61.91 61.91
    lon_u       (eta_u, xi_u) float64 -19.75 -19.51 -19.26 ... 9.262 9.508 9.754
    lat_u       (eta_u, xi_u) float64 35.95 35.95 35.95 ... 61.91 61.91 61.91
    lon_v       (eta_v, xi_v) float64 -19.88 -19.63 -19.39 ... 9.385 9.631 9.877
    lat_v       (eta_v, xi_v) float64 36.1 36.1 36.1 36.1 ... 61.82 61.82 61.82
    ...          ...
    salt        (ocean_time, s_rho, eta_rho, xi_rho) float32 ...
    temp        (ocean_time, s_rho, eta_rho, xi_rho) float32 ...
    zeta        (ocean_time, eta_rho, xi_rho) float32 -0.2964 -0.2781 ... nan
    ubar        (ocean_time, eta_u, xi_u) float32 -0.04235 -0.01933 ... nan nan
    vbar        (ocean_time, eta_v, xi_v) float32 0.002452 -0.01216 ... nan nan
    Cs_rho      (s_rho) float64 -0.9614 -0.8737 ... -0.0002615 -2.879e-05
Attributes:
    title:        Initial forcing file (INIT) used for forcing the ROMS model
    description:  Created for grid file: NORTH_SEA4
    grd_file:     Gridfile: .../NorthSea4_smooth01_sponge_nudg.nc
    history:      Created December 19, 2022

In [26]:
def roms_to_latlon_u(data):
    '''
    Function to obtain a new dataset for the roms data where dimensions are latitude and longitude 
    data should be the ROMS output dataset 
    '''
    
    data_vars = dict(u=(["time", "depth", "latitude", "longitude"], data.u.values),)
        
    coords = dict(time=(["time"], data.ocean_time.values),
                  depth=(["depth"], data.s_rho.values),
             latitude=(["latitude"], data.lat_u.values[:,0]),
             longitude=(["longitude"], data.lon_u.values[0]))
    
    return xr.Dataset(data_vars = data_vars, coords = coords)

def roms_to_latlon_v(data):
    '''
    Function to obtain a new dataset for the roms data where dimensions are latitude and longitude 
    data should be the ROMS output dataset 
    '''
    
    data_vars = dict(v=(["time", "depth", "latitude", "longitude"], data.v.values),)
        
    coords = dict(time=(["time"], data.ocean_time.values),
                  depth=(["depth"], data.s_rho.values),
             latitude=(["latitude"], data.lat_v.values[:,0]),
             longitude=(["longitude"], data.lon_v.values[0]))
    
    return xr.Dataset(data_vars = data_vars, coords = coords)

In [27]:
u = roms_to_latlon_u(data)
v = roms_to_latlon_v(data)

In [28]:
speed = np.sqrt(u**2 + v**2)

In [29]:
speed

<xarray.Dataset>
Dimensions:    (latitude: 0, longitude: 0, time: 1, depth: 30)
Coordinates:
  * latitude   (latitude) float64 
  * longitude  (longitude) float64 
  * time       (time) datetime64[ns] 1993-01-15
  * depth      (depth) float64 -0.9833 -0.95 -0.9167 ... -0.08333 -0.05 -0.01667
Data variables:
    *empty*

In [9]:
speed.s_rho

<xarray.DataArray 's_rho' (s_rho: 30)>
array([-0.983333, -0.95    , -0.916667, -0.883333, -0.85    , -0.816667,
       -0.783333, -0.75    , -0.716667, -0.683333, -0.65    , -0.616667,
       -0.583333, -0.55    , -0.516667, -0.483333, -0.45    , -0.416667,
       -0.383333, -0.35    , -0.316667, -0.283333, -0.25    , -0.216667,
       -0.183333, -0.15    , -0.116667, -0.083333, -0.05    , -0.016667])
Coordinates:
  * s_rho    (s_rho) float64 -0.9833 -0.95 -0.9167 ... -0.08333 -0.05 -0.01667
Attributes:
    long_name:      S-coordinate at RHO-points
    valid_min:      -1.0
    valid_max:      0.0
    standard_name:  ocean_s_coordinate_g2
    formula terms:  s: s_rho C: Cs_r eta: zeta depth: h depth_c: hc
    field:          s_rho, scalar

In [11]:
speed.isel(s_rho = 0)#.plot()

<xarray.DataArray (ocean_time: 1, eta_u: 110, xi_u: 121, eta_v: 109, xi_v: 122)>
array([[[[[0.01652554, 0.01688126, 0.01362803, ...,        nan,
                  nan,        nan],
          [0.00819014, 0.01313237, 0.02789149, ...,        nan,
                  nan,        nan],
          [0.02392924, 0.01703816, 0.01756092, ...,        nan,
                  nan,        nan],
          ...,
          [0.01138633, 0.01832316, 0.03125863, ...,        nan,
                  nan,        nan],
          [0.01268834, 0.00765243, 0.00820275, ...,        nan,
                  nan,        nan],
          [0.02326619, 0.01768008, 0.01510892, ...,        nan,
                  nan,        nan]],

         [[0.02121935, 0.02149753, 0.01904968, ...,        nan,
                  nan,        nan],
          [0.01562835, 0.01869829, 0.03090473, ...,        nan,
                  nan,        nan],
          [0.02738203, 0.02162096, 0.02203527, ...,        nan,
                  nan,        nan],
...
          [       nan,        nan,        nan, ...,        nan,
                  nan,        nan],
          [       nan,        nan,        nan, ...,        nan,
                  nan,        nan],
          [       nan,        nan,        nan, ...,        nan,
                  nan,        nan]],

         [[       nan,        nan,        nan, ...,        nan,
                  nan,        nan],
          [       nan,        nan,        nan, ...,        nan,
                  nan,        nan],
          [       nan,        nan,        nan, ...,        nan,
                  nan,        nan],
          ...,
          [       nan,        nan,        nan, ...,        nan,
                  nan,        nan],
          [       nan,        nan,        nan, ...,        nan,
                  nan,        nan],
          [       nan,        nan,        nan, ...,        nan,
                  nan,        nan]]]]], dtype=float32)
Coordinates:
    s_rho       float64 -0.9833
  * ocean_time  (ocean_time) datetime64[ns] 1993-01-15
Dimensions without coordinates: eta_u, xi_u, eta_v, xi_v